In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  # Pour télécharger et gérer ChromeDriver automatiquement
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os
import shutil
import time
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import requests

In [ ]:
import os
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configuration du navigateur avec undetected-chromedriver
options = uc.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--start-maximized")

# Liste de User-Agents pour rotation
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")

# Lancer le navigateur
driver = uc.Chrome(options=options)

# Fonction pour reformater les dates
MOIS_FRANCAIS = {
    "janv.": "01", "févr.": "02", "mars": "03", "avr.": "04", "mai": "05", "juin": "06",
    "juil.": "07", "août": "08", "sept.": "09", "oct.": "10", "nov.": "11", "déc.": "12"
}


def reformater_date(date_str):
    parts = date_str.split()
    jour = parts[0]
    mois = MOIS_FRANCAIS[parts[1]]
    annee = parts[2]
    return f"{jour}/{mois}/{annee}"


# Fonction pour défiler la page pour simuler un comportement naturel
def scroll_page():
    for _ in range(random.randint(2, 5)):
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(random.uniform(1, 3))


# Fonction pour simuler des clics
def simulate_click(element):
    action = ActionChains(driver)
    action.move_to_element(element).click().perform()
    time.sleep(random.uniform(2, 4))


# ... Le reste du script reste identique ...

# Fonction pour scraper une page
def scrape_page(page_url):
    driver.get(page_url)
    time.sleep(random.uniform(5, 7))  # Pause aléatoire
    scroll_page()  # Simuler un défilement
    soup = BeautifulSoup(driver.page_source, "html.parser")
    rows = soup.select("tr.annentry-line.deals, tr.annentry-line.deals.pyjama")

    page_data = []
    for row in rows:
        # Récupérer les données
        date = row.select_one("div.ezdatetime-field")
        date_text = date.text.strip() if date else "N/A"

        target = row.select_one("td.cible")
        target_name = target.select_one("i.bold700").text.strip() if target and target.select_one("i.bold700") else "N/A"
        target_tags = [tag.text.strip() for tag in target.select("span.tag")] if target else []
        target_description = target.select_one("a.activity-target")
        target_description_text = (
            target_description.select_one("span.tooltiptext").text.strip()
            if target_description and target_description.select_one("span.tooltiptext")
            else "N/A"
        )

        revenue_td = target.find_next("td", class_="text-center footable-visible")
        revenue_text = revenue_td.contents[0].strip() if revenue_td and revenue_td.contents else "N/A"
        revenue_year = revenue_td.select_one("span.ezinteger-field")
        revenue_year_text = revenue_year.text.strip() if revenue_year else "N/A"

        sector = row.select_one("td[data-class='sector']")
        sector_text = (
            sector.select_one("span.tooltiptext-sector").text.strip()
            if sector and sector.select_one("span.tooltiptext-sector")
            else "N/A"
        )

        buyer = row.select_one("td .inv-div")
        buyers = [b.text.strip() for b in buyer.select("a")] if buyer else []

        seller = row.select_one("td.desktop.border-left.footable-visible")
        seller_text = seller.text.strip() if seller else "N/A"

        valorisation = row.select_one("td.text-center.bg-lowlight-grey.footable-visible")
        valorisation_text = valorisation.text.strip() if valorisation else "N/A"

        operation = row.select_one("td.desktop.px-1.footable-visible span.operation_subtype")
        operation_text = operation.text.strip() if operation else "N/A"

        transaction_td = revenue_td.find_next("td", class_="text-center footable-visible")
        transaction_amount = transaction_td.select_one("div.ezselection-field")
        transaction_amount_text = transaction_amount.text.strip() if transaction_amount else "N/A"

        row_data = {
            "Date": date_text,
            "Target Name": target_name,
            "Target Tags": ", ".join(target_tags),
            "Target Description": target_description_text,
            "Revenue": revenue_text,
            "Revenue Year": revenue_year_text,
            "Sector": sector_text,
            "Buyers": ", ".join(buyers),
            "Sellers": seller_text,
            "Valorisation": valorisation_text,
            "Operation Type": operation_text,
            "Transaction Amount": transaction_amount_text,
        }
        page_data.append(row_data)
        print(f"Ligne ajoutée : {row_data}")  # Affichage de la ligne récupérée
    return page_data


from datetime import datetime

# Fonction pour convertir une date en objet datetime
def convertir_date(date_str):
    try:
        date_parts = date_str.split()
        jour = int(date_parts[0])
        mois = MOIS_FRANCAIS[date_parts[1]]
        annee = int(date_parts[2])
        return datetime.strptime(f"{jour}/{mois}/{annee}", "%d/%m/%Y")
    except Exception as e:
        print(f"Erreur lors de la conversion de la date : {date_str} - {e}")
        return None

# Script principal
try:
    # Connexion
    driver.get("https://www.cfnews.net/cfn/login/?redir=/Annuaires-base-de-deals/Base-de-deals&domain=www.cfnews.net")
    time.sleep(random.uniform(5, 8))  # Pause aléatoire
    email_field = driver.find_element(By.NAME, "Login")
    email_field.send_keys("namar@da-cf.fr")
    password_field = driver.find_element(By.NAME, "Password")
    password_field.send_keys("Noeamar2209#")
    login_button = driver.find_element(By.NAME, "LoginButton")
    simulate_click(login_button)  # Clic simulé
    time.sleep(random.uniform(5, 8))  # Pause aléatoire

    base_url = "https://www.cfnews.net/Annuaires-base-de-deals/Base-de-deals?locationId=93&sort_attribute=fiche_operation_operation_date_value_dt&sort_type=descending&rechercher=Rechercher&depuis=01%2F01%2F2019&jusquau={}"
    jusquau_date = "02/01/2025"
    output_file = "deals_data_filtered_pagination.csv"
    all_data = []

    # Charger les données existantes
    if os.path.exists(output_file):
        existing_data = pd.read_csv(output_file)
    else:
        existing_data = pd.DataFrame()

    # Date de fin
    date_fin = convertir_date("20 dec. 2024")

    while True:
        for page_num in range(1, 18):
            page_url = base_url.format(jusquau_date) if page_num == 1 else f"{base_url.format(jusquau_date)}&page={page_num}"
            page_data = scrape_page(page_url)
            all_data += page_data

        # Trouver la dernière date récupérée
        dates = [deal["Date"] for deal in all_data if deal["Date"] != "N/A"]
        if not dates:
            break
        last_date_str = dates[-1]
        last_date = convertir_date(last_date_str)

        if last_date and last_date <= date_fin:
            print(f"Condition d'arrêt atteinte : dernière date récupérée = {last_date_str}")
            break

        # Mettre à jour "jusqu'au" avec la dernière date récupérée
        jusquau_date = last_date.strftime("%d/%m/%Y").replace("/", "%2F") if last_date else jusquau_date

    # Combiner les données et supprimer les doublons
    new_data = pd.DataFrame(all_data)
    combined_data = pd.concat([existing_data, new_data]).drop_duplicates().reset_index(drop=True)

    # Enregistrer les données mises à jour
    combined_data.to_csv(output_file, index=False)
    print("Données mises à jour enregistrées.")
finally:
    driver.quit()


### Scraping jusqu'à 2019

In [ ]:
import os
import random
import csv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

# Configuration du navigateur avec undetected-chromedriver
options = uc.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--start-maximized")

# Liste de User-Agents pour rotation
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")

# Lancer le navigateur
driver = uc.Chrome(options=options)

# Fonction pour reformater les dates
MOIS_FRANCAIS = {
    "janv.": "01", "févr.": "02", "mars": "03", "avr.": "04", "mai": "05", "juin": "06",
    "juil.": "07", "août": "08", "sept.": "09", "oct.": "10", "nov.": "11", "déc.": "12"
}

def reformater_date(date_str):
    parts = date_str.split()
    jour = parts[0]
    mois = MOIS_FRANCAIS[parts[1]]
    annee = parts[2]
    return f"{jour}/{mois}/{annee}"

def convertir_date(date_str):
    try:
        date_parts = date_str.split()
        jour = int(date_parts[0])
        mois = MOIS_FRANCAIS[date_parts[1]]
        annee = int(date_parts[2])
        return datetime.strptime(f"{jour}/{mois}/{annee}", "%d/%m/%Y")
    except Exception as e:
        print(f"Erreur lors de la conversion de la date : {date_str} - {e}")
        return None

def scroll_page():
    for _ in range(random.randint(2, 5)):
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(random.uniform(1, 3))

def simulate_click(element):
    action = ActionChains(driver)
    action.move_to_element(element).click().perform()
    time.sleep(random.uniform(2, 4))

# Fonction pour initialiser le fichier CSV
def initialize_csv(file_path):
    if not os.path.exists(file_path):
        with open(file_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=[
                "Date", "Target Name", "Target Tags", "Target Description", "Revenue", 
                "Revenue Year", "Sector", "Buyers", "Sellers", 
                "Valorisation", "Operation Type", "Transaction Amount"
            ])
            writer.writeheader()

# Fonction pour ajouter une ligne au CSV
def append_to_csv(file_path, row_data):
    with open(file_path, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[
            "Date", "Target Name", "Target Tags", "Target Description", "Revenue", 
            "Revenue Year", "Sector", "Buyers", "Sellers", 
            "Valorisation", "Operation Type", "Transaction Amount"
        ])
        writer.writerow(row_data)

# Fonction pour scraper une page
def scrape_page(page_url, output_file, existing_data):
    driver.get(page_url)
    time.sleep(random.uniform(5, 7))
    scroll_page()
    soup = BeautifulSoup(driver.page_source, "html.parser")
    rows = soup.select("tr.annentry-line.deals, tr.annentry-line.deals.pyjama")

    for row in rows:
        date = row.select_one("div.ezdatetime-field")
        date_text = date.text.strip() if date else "N/A"

        target = row.select_one("td.cible")
        target_name = target.select_one("i.bold700").text.strip() if target and target.select_one("i.bold700") else "N/A"
        target_tags = [tag.text.strip() for tag in target.select("span.tag")] if target else []
        target_description = target.select_one("a.activity-target")
        target_description_text = (
            target_description.select_one("span.tooltiptext").text.strip()
            if target_description and target_description.select_one("span.tooltiptext")
            else "N/A"
        )

        revenue_td = target.find_next("td", class_="text-center footable-visible")
        revenue_text = revenue_td.contents[0].strip() if revenue_td and revenue_td.contents else "N/A"
        revenue_year = revenue_td.select_one("span.ezinteger-field")
        revenue_year_text = revenue_year.text.strip() if revenue_year else "N/A"

        sector = row.select_one("td[data-class='sector']")
        sector_text = (
            sector.select_one("span.tooltiptext-sector").text.strip()
            if sector and sector.select_one("span.tooltiptext-sector")
            else "N/A"
        )

        buyer = row.select_one("td .inv-div")
        buyers = [b.text.strip() for b in buyer.select("a")] if buyer else []

        seller = row.select_one("td.desktop.border-left.footable-visible")
        seller_text = seller.text.strip() if seller else "N/A"

        valorisation = row.select_one("td.text-center.bg-lowlight-grey.footable-visible")
        valorisation_text = valorisation.text.strip() if valorisation else "N/A"

        operation = row.select_one("td.desktop.px-1.footable-visible span.operation_subtype")
        operation_text = operation.text.strip() if operation else "N/A"

        transaction_td = revenue_td.find_next("td", class_="text-center footable-visible")
        transaction_amount = transaction_td.select_one("div.ezselection-field")
        transaction_amount_text = transaction_amount.text.strip() if transaction_amount else "N/A"

        row_data = {
            "Date": date_text,
            "Target Name": target_name,
            "Target Tags": ", ".join(target_tags),
            "Target Description": target_description_text,
            "Revenue": revenue_text,
            "Revenue Year": revenue_year_text,
            "Sector": sector_text,
            "Buyers": ", ".join(buyers),
            "Sellers": seller_text,
            "Valorisation": valorisation_text,
            "Operation Type": operation_text,
            "Transaction Amount": transaction_amount_text,
        }

        # Vérifier si les données existent déjà
        if row_data not in existing_data:
            append_to_csv(output_file, row_data)
            existing_data.append(row_data)
            print(f"Ligne ajoutée : {row_data}")

# Script principal
try:
    # Démarrer un chronomètre pour limiter l'exécution à 5 minutes
    start_time = time.time()
    max_duration = 3 * 60  # 5 minutes en secondes

    driver.get("https://www.cfnews.net/cfn/login/?redir=/Annuaires-base-de-deals/Base-de-deals&domain=www.cfnews.net")
    time.sleep(random.uniform(5, 8))
    
    # Connexion
    email_field = driver.find_element(By.NAME, "Login")
    email_field.send_keys("namar@da-cf.fr")
    password_field = driver.find_element(By.NAME, "Password")
    password_field.send_keys("Noeamar2209#")
    login_button = driver.find_element(By.NAME, "LoginButton")
    simulate_click(login_button)
    time.sleep(random.uniform(5, 8))

    # Initialiser le fichier de sortie
    output_file = "deals_data_filtered_pagination.csv"
    initialize_csv(output_file)

    # Charger les données existantes pour éviter les doublons
    existing_data = []
    if os.path.exists(output_file):
        existing_data = pd.read_csv(output_file).to_dict("records")

    base_url = "https://www.cfnews.net/Annuaires-base-de-deals/Base-de-deals?locationId=93&sort_attribute=fiche_operation_operation_date_value_dt&sort_type=descending&rechercher=Rechercher&depuis=01%2F01%2F2019&jusquau={}"

    # Trouver la date la plus ancienne dans le fichier existant
    if existing_data:
        oldest_date = min(
            [convertir_date(row["Date"]) for row in existing_data if row["Date"] != "N/A"],
            default=None
        )
        jusquau_date = oldest_date.strftime("%d/%m/%Y").replace("/", "%2F") if oldest_date else "01/01/2025"
    else:
        jusquau_date = "01/01/2025"

    # Date de fin pour arrêter le scraping
    date_fin = convertir_date("01 janv. 2025")

    while True:
        for page_num in range(1, 18):
            # Vérifier si le temps limite est dépassé
            elapsed_time = time.time() - start_time
            if elapsed_time > max_duration:
                print("Temps limite de 3 minutes atteint. Arrêt du scraping.")
                break

            # Construire l'URL et scraper la page
            page_url = base_url.format(jusquau_date) if page_num == 1 else f"{base_url.format(jusquau_date)}&page={page_num}"
            scrape_page(page_url, output_file, existing_data)

        # Vérifier si le temps limite est dépassé après une boucle
        elapsed_time = time.time() - start_time
        if elapsed_time > max_duration:
            print("Temps limite de 3 minutes atteint. Arrêt du scraping.")
            break

        # Mettre à jour les dates existantes après le scraping
        dates = [convertir_date(row["Date"]) for row in existing_data if row["Date"] != "N/A"]
        if not dates:
            break
        last_date = min(dates)

        # Condition d'arrêt
        if last_date and last_date <= date_fin:
            print(f"Condition d'arrêt atteinte : dernière date récupérée = {last_date.strftime('%d/%m/%Y')}")
            break

        # Mettre à jour `jusquau_date` pour la prochaine itération
        jusquau_date = last_date.strftime("%d/%m/%Y").replace("/", "%2F") if last_date else jusquau_date

        # Charger les données existantes mises à jour après chaque page
        existing_data = pd.read_csv(output_file).to_dict("records")

finally:
    driver.quit()
    print("Scraping terminé.")



Ligne ajoutée : {'Date': '30 août 2023', 'Target Name': 'PLUSMETA', 'Target Tags': 'Allemagne |, Etranger', 'Target Description': "Fournisseur\xa0de logiciels cloud basés sur l'IA", 'Revenue': '', 'Revenue Year': 'N/A', 'Sector': 'Logiciel et services informatiques', 'Buyers': 'QUANOS', 'Sellers': 'MANAGER(S)', 'Valorisation': 'n.d.', 'Operation Type': 'Build-up international', 'Transaction Amount': 'n.d.'}
Ligne ajoutée : {'Date': '29 août 2023', 'Target Name': 'SUSU', 'Target Tags': 'France |, Île-de-France', 'Target Description': "Services de santé et d'assurance destinés aux familles de la diaspora africaine", 'Revenue': '', 'Revenue Year': 'N/A', 'Sector': 'Internet & ecommerce, eservices', 'Buyers': 'AL MADA, JANNGO CAPITAL, INCO VENTURES (EX LE CO..., OPEN CNP, HEALTH54, LAUNCH AFRICA VENTURES, FIVE35 VENTURES, PLUG AND PLAY TECH CENTER', 'Sellers': 'n/a', 'Valorisation': '8.90 M€', 'Operation Type': '1er tour', 'Transaction Amount': 'N/A'}
Ligne ajoutée : {'Date': '29 août 2023

### Actualisation du scraping

In [2]:
import os
import random
import csv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

# Configuration du navigateur avec undetected-chromedriver
options = uc.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--start-maximized")

# Liste de User-Agents pour rotation
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")

# Lancer le navigateur
driver = uc.Chrome(options=options)

# Fonction pour reformater les dates
MOIS_FRANCAIS = {
    "janv.": "01", "févr.": "02", "mars": "03", "avr.": "04", "mai": "05", "juin": "06",
    "juil.": "07", "août": "08", "sept.": "09", "oct.": "10", "nov.": "11", "déc.": "12"
}

def reformater_date(date_str):
    parts = date_str.split()
    if len(parts) < 3:
        return "N/A"
    jour = parts[0]
    mois = MOIS_FRANCAIS.get(parts[1].lower(), "01")  # Défaut à janvier si mois inconnu
    annee = parts[2]
    return f"{jour}/{mois}/{annee}"

def convertir_date(date_str):
    try:
        parts = date_str.split()
        if len(parts) < 3:
            raise ValueError("Format de date incorrect")
        jour = int(parts[0])
        mois = MOIS_FRANCAIS.get(parts[1].lower(), "01")
        annee = int(parts[2])
        return datetime.strptime(f"{jour}/{mois}/{annee}", "%d/%m/%Y")
    except Exception as e:
        print(f"Erreur lors de la conversion de la date : {date_str} - {e}")
        return None

def scroll_page():
    for _ in range(random.randint(2, 5)):
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(random.uniform(1, 3))

def simulate_click(element):
    action = ActionChains(driver)
    action.move_to_element(element).click().perform()
    time.sleep(random.uniform(2, 4))

# Fonction pour initialiser le fichier CSV
def initialize_csv(file_path):
    if not os.path.exists(file_path):
        with open(file_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=[
                "Date", "Target Name", "Target Tags", "Target Description", "Revenue", 
                "Revenue Year", "Sector", "Buyers", "Sellers", 
                "Valorisation", "Operation Type", "Transaction Amount"
            ])
            writer.writeheader()

# Fonction pour ajouter une ligne au CSV
def append_to_csv(file_path, row_data):
    with open(file_path, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[
            "Date", "Target Name", "Target Tags", "Target Description", "Revenue", 
            "Revenue Year", "Sector", "Buyers", "Sellers", 
            "Valorisation", "Operation Type", "Transaction Amount"
        ])
        writer.writerow(row_data)

# Fonction pour scraper une page
def scrape_page(page_url, output_file, existing_ids, oldest_date):
    driver.get(page_url)
    time.sleep(random.uniform(5, 7))
    scroll_page()
    soup = BeautifulSoup(driver.page_source, "html.parser")
    rows = soup.select("tr.annentry-line.deals, tr.annentry-line.deals.pyjama")

    for row in rows:
        # Extraction des données de chaque colonne
        date = row.select_one("div.ezdatetime-field")
        date_text = date.text.strip() if date else "N/A"

        target = row.select_one("td.cible")
        target_name = target.select_one("i.bold700").text.strip() if target and target.select_one("i.bold700") else "N/A"
        target_tags = [tag.text.strip() for tag in target.select("span.tag")] if target else []
        target_description = target.select_one("a.activity-target")
        target_description_text = (
            target_description.select_one("span.tooltiptext").text.strip()
            if target_description and target_description.select_one("span.tooltiptext")
            else "N/A"
        )

        revenue_td = target.find_next("td", class_="text-center footable-visible") if target else None
        revenue_text = revenue_td.contents[0].strip() if revenue_td and revenue_td.contents else "N/A"
        revenue_year = revenue_td.select_one("span.ezinteger-field").text.strip() if revenue_td and revenue_td.select_one("span.ezinteger-field") else "N/A"

        sector = row.select_one("td[data-class='sector']")
        sector_text = (
            sector.select_one("span.tooltiptext-sector").text.strip()
            if sector and sector.select_one("span.tooltiptext-sector")
            else "N/A"
        )

        buyer = row.select_one("td .inv-div")
        buyers = [b.text.strip() for b in buyer.select("a")] if buyer else []

        seller = row.select_one("td.desktop.border-left.footable-visible")
        seller_text = seller.text.strip() if seller else "N/A"

        valorisation = row.select_one("td.text-center.bg-lowlight-grey.footable-visible")
        valorisation_text = valorisation.text.strip() if valorisation else "N/A"

        operation = row.select_one("td.desktop.px-1.footable-visible span.operation_subtype")
        operation_text = operation.text.strip() if operation else "N/A"

        transaction_td = revenue_td.find_next("td", class_="text-center footable-visible") if revenue_td else None
        transaction_amount = transaction_td.select_one("div.ezselection-field") if transaction_td else None
        transaction_amount_text = transaction_amount.text.strip() if transaction_amount else "N/A"

        row_data = {
            "Date": date_text,
            "Target Name": target_name,
            "Target Tags": ", ".join(target_tags),
            "Target Description": target_description_text,
            "Revenue": revenue_text,
            "Revenue Year": revenue_year,
            "Sector": sector_text,
            "Buyers": ", ".join(buyers),
            "Sellers": seller_text,
            "Valorisation": valorisation_text,
            "Operation Type": operation_text,
            "Transaction Amount": transaction_amount_text,
        }

        # Création d'un identifiant unique pour le deal (par exemple, combinaison de Date et Target Name)
        deal_id = f"{row_data['Date']}_{row_data['Target Name']}"

        # Vérifier si le deal a déjà été scrappé
        if deal_id in existing_ids:
            print(f"Deal déjà scrappé trouvé : {deal_id}. Arrêt du scraping.")
            return False  # Indique d'arrêter le scraping

        # Convertir la date pour comparaison
        date_obj = convertir_date(date_text)
        if date_obj is None:
            continue  # Ignorer les deals avec une date invalide

        if date_obj <= oldest_date:
            print(f"Deal atteint la date la plus ancienne enregistrée : {date_text}. Ajout et arrêt.")
            append_to_csv(output_file, row_data)
            existing_ids.add(deal_id)
            return False  # Arrêter après avoir ajouté ce deal

        # Ajouter les nouvelles données au CSV immédiatement
        append_to_csv(output_file, row_data)
        existing_ids.add(deal_id)
        print(f"Ligne ajoutée : {row_data}")

    return True  # Indique de continuer le scraping

# Script principal
try:
    # Démarrer un chronomètre pour limiter l'exécution à 5 minutes
    start_time = time.time()
    max_duration = 5 * 60  # 5 minutes en secondes
    
    # Naviguer vers la page de connexion
    driver.get("https://www.cfnews.net/cfn/login/?redir=/Annuaires-base-de-deals/Base-de-deals&domain=www.cfnews.net")
    time.sleep(random.uniform(5, 8))
    
    # Connexion
    email_field = driver.find_element(By.NAME, "Login")
    email_field.send_keys("namar@da-cf.fr")
    password_field = driver.find_element(By.NAME, "Password")
    password_field.send_keys("Noeamar2209#")
    login_button = driver.find_element(By.NAME, "LoginButton")
    simulate_click(login_button)
    time.sleep(random.uniform(5, 8))

    # Initialiser le fichier de sortie
    output_file = "deals_data_filtered_pagination.csv"
    initialize_csv(output_file)

    # Charger les données existantes pour éviter les doublons
    existing_ids = set()
    oldest_date = None
    if os.path.exists(output_file):
        existing_data = pd.read_csv(output_file)
        # Créer un ensemble d'identifiants uniques pour les vérifications rapides
        existing_ids = set(f"{row['Date']}_{row['Target Name']}" for index, row in existing_data.iterrows())
        # Trouver la date la plus ancienne
        existing_dates = existing_data['Date'].apply(convertir_date).dropna()
        oldest_date = existing_dates.min() if not existing_dates.empty else None
    else:
        # Si le fichier n'existe pas, définir une date très ancienne
        oldest_date = convertir_date("01 janv. 2019")
        print("Aucune donnée existante. Scraping depuis le début.")

    if oldest_date:
        print(f"Date la plus ancienne dans le CSV : {oldest_date.strftime('%d/%m/%Y')}")
    else:
        # Si pour une raison quelconque, la date la plus ancienne n'a pas pu être déterminée
        oldest_date = convertir_date("01 janv. 2019")
        print("Date la plus ancienne non trouvée. Utilisation de la date par défaut : 01/01/2019")

    # Définir jusquau_date comme la date d'aujourd'hui
    today = datetime.today()
    jusquau_date = today.strftime("%d/%m/%Y").replace("/", "%2F")
    print(f"Date de début pour le scraping : {today.strftime('%d/%m/%Y')}")

    base_url = "https://www.cfnews.net/Annuaires-base-de-deals/Base-de-deals?locationId=93&sort_attribute=fiche_operation_operation_date_value_dt&sort_type=descending&rechercher=Rechercher&depuis=01%2F01%2F2019&jusquau={}"

    while True:
        for page_num in range(1, 19):  # Pages 1 à 18 incluses
            # Vérifier si le temps limite est dépassé
            elapsed_time = time.time() - start_time
            if elapsed_time > max_duration:
                print("Temps limite de 5 minutes atteint. Arrêt du scraping.")
                raise TimeoutError("Temps limite atteint")

            # Construire l'URL et scraper la page
            if page_num == 1:
                # Utiliser la date d'aujourd'hui pour la première page
                page_url = base_url.format(jusquau_date)
            else:
                page_url = f"{base_url.format(jusquau_date)}&page={page_num}"
            print(f"Scraping de la page {page_num} : {page_url}")
            continuer = scrape_page(page_url, output_file, existing_ids, oldest_date)

            if not continuer:
                break  # Sortir de la boucle des pages si condition d'arrêt atteinte

        # Vérifier si le temps limite est dépassé après une boucle
        elapsed_time = time.time() - start_time
        if elapsed_time > max_duration:
            print("Temps limite de 5 minutes atteint. Arrêt du scraping.")
            break

        # Sortir de la boucle principale si la condition d'arrêt a été atteinte
        if not continuer:
            print("Condition d'arrêt atteinte. Arrêt du scraping.")
            break

        # Mettre à jour les dates existantes après le scraping
        if os.path.exists(output_file):
            existing_data = pd.read_csv(output_file)
            existing_dates = existing_data['Date'].apply(convertir_date).dropna()
            if not existing_dates.empty:
                oldest_date = existing_dates.min()
                print(f"Mise à jour de la date la plus ancienne : {oldest_date.strftime('%d/%m/%Y')}")
            else:
                break
        else:
            break

except TimeoutError as te:
    print(te)
except Exception as e:
    print(f"Une erreur est survenue : {e}")
finally:
    driver.quit()
    print("Scraping terminé.")


Date la plus ancienne dans le CSV : 10/10/2023
Date de début pour le scraping : 10/02/2025
Scraping de la page 1 : https://www.cfnews.net/Annuaires-base-de-deals/Base-de-deals?locationId=93&sort_attribute=fiche_operation_operation_date_value_dt&sort_type=descending&rechercher=Rechercher&depuis=01%2F01%2F2019&jusquau=10%2F02%2F2025
Ligne ajoutée : {'Date': '07 févr. 2025', 'Target Name': 'PARTI PRIS', 'Target Tags': 'France |, Île-de-France', 'Target Description': 'Agence de communication événementielle', 'Revenue': '20', 'Revenue Year': '2024', 'Sector': 'Média, Marketing/Communication et Télécom', 'Buyers': 'AUDITOIRE', 'Sellers': 'PARTI PRIS\n                                                                                    \n                            ,                                                                                               \n\n\n                                                                        FONDATEUR(S)\n                    \n                       

### Configuration du fichier

In [6]:
import pandas as pd

# Charger le fichier CSV
data_file = "C:\\Users\\namar\\Documents\\poc_RAG\\Projet_test\\RAG_MnA\\deals_data_filtered_pagination.csv"
data = pd.read_csv(data_file)

# Nettoyer les espaces inutiles dans la colonne 'Sellers'
data['Sellers'] = data['Sellers'].apply(lambda x: ' '.join(str(x).split()) if pd.notnull(x) else x)

# Ajouter les noms de colonnes devant les valeurs pour chaque ligne
def prefix_columns(row):
    return ', '.join([f"{col}: {row[col]}" for col in row.index])

# Appliquer la transformation ligne par ligne
data['Formatted_Row'] = data.apply(prefix_columns, axis=1)

# Extraire uniquement les lignes formatées dans une nouvelle colonne
formatted_data = data[['Formatted_Row']]

# Sauvegarder le fichier formaté
output_file = "C:\\Users\\namar\\Documents\\poc_RAG\\Projet_test\\RAG_MnA\\Data\\deals_data_cleaned_CFNews.csv"
formatted_data.to_csv(output_file, index=False, header=False, encoding='utf-8')

print(f"Fichier formaté sauvegardé sous : {output_file}")

Fichier formaté sauvegardé sous : C:\Users\namar\Documents\poc_RAG\Projet_test\RAG_MnA\Data\deals_data_cleaned_CFNews.csv
